# 네이버 뉴스 크롤링 - Graph DB 데이터 수집

네이버 뉴스에서 분야별 헤드라인 기사 수집

- 경제: https://news.naver.com/section/101
- IT/과학: https://news.naver.com/section/105

## 수집 항목 (Articles.xlsx)
- article_id: 기사 고유 id
- title: 기사 제목
- content: 기사 본문
- url: 기사 url
- published_date: 발행일
- source: 언론사
- author: 기자명
- category: 카테고리

## 1. 필요한 패키지 설치 및 임포트

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from datetime import datetime
import re

## 2. 카테고리 설정 및 Selenium 초기화

In [2]:
categories = {
    '경제': 'https://news.naver.com/section/101',
    'IT/과학': 'https://news.naver.com/section/105'
}

NUM_ARTICLES_PER_CATEGORY = 10

In [3]:
service = Service(ChromeDriverManager().install())              # 설치돼있는 크롬을 불러온다.
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')                            
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=service, options=options)

## 3. 기사 수집 함수 정의

In [4]:
def get_article_links(driver, category_url, num_articles):
    driver.get(category_url)
    time.sleep(3)

    article_links = []

    try:
        selectors = [
            'a.sa_text_lede',
            'a.sa_text_strong',
            '.sa_text a',
            '.cluster_text_headline a',
            '.cluster_text_lede a'
        ]

        for selector in selectors:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                url = element.get_attribute('href')
                if (url and 'news.naver.com' in url and '/article/' in url
                    and '/comment/' not in url  # 댓글 페이지만 제외
                    and url not in article_links):
                    print('통과')
                    article_links.append(url)
                    if len(article_links) >= num_articles:
                        break
            if len(article_links) >= num_articles:
                break
        print(f'✅ {len(article_links)}개의 기사 링크 수집 완료')
    except Exception as e:
        print(f'❎ 기사 링크 수집 실패: {e}')
    
    return article_links[:num_articles]

In [5]:
def parse_article_detail(driver, article_url, category):
    driver.get(article_url)
    time.sleep(1.5)

    article_data = {
        'article_id': '',
        'title': '',
        'content': '',
        'url': article_url,
        'published_date': '',
        'source': '',
        'author': '',
        'category': category
    }

    try:
        # 기사 ID 생성 (url에서 추출)
        match = re.search(r'article/(₩d+)/(₩d+)', article_url)
        if match:
            article_data['article_id'] = f"ART_{match.group(1)}_{match.group(2)}"
        else:
            article_data['article_id'] = f"ART_{datetime.now().strftime('%Y%m%d%H%M%S')}"
        
        # 제목
        title_selectors = [
            '#title_area span',
            '#ct .media_end_head_headline',
            '.media_end_head_headline',
            'h2#title_area',
            '.news_end_title'
        ]

        for selector in title_selectors:
            try:
                title_element = driver.find_element(By.CSS_SELECTOR, selector)
                if title_element.text.strip():
                    article_data['title'] = title_element.text.strip()
            except:
                continue

        # 본문
        content_selectors = [
            '#dic_area',
            'article#dic_area',
            '.go_trans._article_content',
            '._article_body_contents'
        ]

        for selector in content_selectors:
            try:
                content_element = driver.find_element(By.CSS_SELECTOR, selector)
                if content_element.text.strip():
                    article_data['content'] = content_element.text.strip()
            except:
                continue
        
        # 언론사
        try:
            source_element = driver.find_element(By.CSS_SELECTOR, 'a.media_end_head_top_logo img')
            article_data['source'] = source_element.get_attribute('alt')
        except:
            try:
                source_element = driver.find_element(By.CSS_SELECTOR, 'media_end_head_top_logo img')
                article_data['source'] = source_element.text.strip()
            except:
                pass
        
        # 발행일
        try:
            date_element = driver.find_element(By.CSS_SELECTOR, '' \
            'span.media_end_head_info_datestamp_time, span[data-date-time]')
            date_text = date_element.get_attribute('data-date-time') or date_element.txt
            article_data['published-date'] = date_text.strip()
        except:
            article_data['published_date'] = datetime.now().strftime('%Y-%m-%d %H:%M')
        
        # 기자명
        try:
            author_element = driver.find_element(By.CSS_SELECTOR, 'em.media_end_head_journalist_name, span.byline_s')
            article_data['author'] = author_element.text.strip()
        except:
            pass
    
    except Exception as e:
        print(f'❎ 파싱 오류: {e}')
    
    return article_data


## 4. 전체 카테고리 크롤링 실행

In [6]:
# 전체 기사 데이터를 저장할 리스트
all_articles = []

# 카테고리별 크롤링
for category_name, category_url in categories.items():
    print(f"\n{'='*60}")
    print(f"[{category_name}] 카테고리 수집 시작...")
    print(f"{'='*60}")

    # 1단계: 기사 링크 수집
    article_links = get_article_links(driver, category_url, NUM_ARTICLES_PER_CATEGORY)

    # 2단계: 각 기사 상세 정보 수집
    for idx, article_url in enumerate(article_links, 1):
        print(f"  [{idx}/{len(article_links)}] {article_url}")
        article_data = parse_article_detail(driver, article_url, category_name)

        if article_data['title']:   # 제목이 있는 경우만 추가
            all_articles.append(article_data)
            print(f"  ✅ 수집 완료:  {article_data['title'][:50]}...")
        else:
            print(f"  ❎ 수집 실패 - 제목을 찾을 수 없습니다.")
        
        time.sleep(0.5)


[경제] 카테고리 수집 시작...
통과
통과
통과
통과
통과
통과
통과
통과
통과
통과
✅ 10개의 기사 링크 수집 완료
  [1/10] https://n.news.naver.com/mnews/article/277/0005718276
  ✅ 수집 완료:  DL이앤씨, 지난해 영업이익 3870억원…전년比 42.8%↑...
  [2/10] https://n.news.naver.com/mnews/article/001/0015888090
  ✅ 수집 완료:  치킨기름 공급사 유통마진 내린 교촌…고법 "과징금 정당"...
  [3/10] https://n.news.naver.com/mnews/article/009/0005633542
  ✅ 수집 완료:  한화시스템, 지난해 매출 3조 돌파…전년 대비 31% 상승...
  [4/10] https://n.news.naver.com/mnews/article/215/0001240775
  ✅ 수집 완료:  비트코인, 날개 없는 추락..."서사의 위기 탓"...
  [5/10] https://n.news.naver.com/mnews/article/005/0001830774
  ✅ 수집 완료:  BNK금융그룹, 순이익 8150억원… 전년 대비 11.9% 증가...
  [6/10] https://n.news.naver.com/mnews/article/119/0003057118
  ✅ 수집 완료:  금호타이어, 작년 영업익 5755억…전년비 2.2%↓...
  [7/10] https://n.news.naver.com/mnews/article/008/0005315103
  ✅ 수집 완료:  '비렉스·글로벌' 양날개 단 코웨이…매출 5조 시대 정조준...
  [8/10] https://n.news.naver.com/mnews/article/001/0015889292
  ✅ 수집 완료:  '방산 빅4' 한화에어로-KAI, 항공엔진·무인기 등 미래사업 맞손...
  [9/10] https://n.news.naver.com/mnews/art

In [7]:
df_articles = pd.DataFrame(all_articles)

In [8]:
df_articles

,article_id,title,content,url,published_date,source,author,category,published-date
0,ART_20260206193839,"DL이앤씨, 지난해 영업이익 3870억원…전년比 42.8%↑",매출 7조4024억원 전년比 11%↓\n외형축소에도 영업이익 호조\nDL이앤씨의 지...,https://n.news.naver.com/mnews/article/277/000...,,아시아경제,이지은 기자,경제,2026-02-06 17:07:56
1,ART_20260206193841,"치킨기름 공급사 유통마진 내린 교촌…고법 ""과징금 정당""","공정위 상대 시정명령 등 취소소송 패소…""거래상 우월한 지위 악용""\n\n\n교촌에...",https://n.news.naver.com/mnews/article/001/001...,,연합뉴스,이도흔 기자,경제,2026-02-05 15:46:11
2,ART_20260206193843,"한화시스템, 지난해 매출 3조 돌파…전년 대비 31% 상승",한화시스템의 2025년 매출이 전년 대비 30.7% 늘어난 3조6641억원을 기록했...,https://n.news.naver.com/mnews/article/009/000...,,매일경제,박승주 기자,경제,2026-02-06 14:30:11
3,ART_20260206193846,"비트코인, 날개 없는 추락...""서사의 위기 탓""",비트코인의 가격이 연이어 추락하고 있다.\n\n미 가상화폐 거래소 코인베이스에 따르...,https://n.news.naver.com/mnews/article/215/000...,,한국경제TV,박근아 기자 twilight1093@wowtv.co.kr,경제,2026-02-06 06:13:52
4,ART_20260206193848,"BNK금융그룹, 순이익 8150억원… 전년 대비 11.9% 증가",비이자이익 확대·대손비용 감소\n비은행 순이익 433억원 증가\n배당성향 28.1%...,https://n.news.naver.com/mnews/article/005/000...,,국민일보,윤일선 기자,경제,2026-02-06 16:52:07
5,ART_20260206193850,"금호타이어, 작년 영업익 5755억…전년비 2.2%↓",매출 4조7013억원으로 3.7% 증가\n금호타이어 홈페이지. 금호타이어 홈페이지 ...,https://n.news.naver.com/mnews/article/119/000...,,데일리안,정진주 기자,경제,2026-02-06 17:06:15
6,ART_20260206193852,'비렉스·글로벌' 양날개 단 코웨이…매출 5조 시대 정조준,코웨이 매출 '5조 클럽' 눈앞/그래픽=이지혜\n\n코웨이가 침대·힐링케어 브랜드 ...,https://n.news.naver.com/mnews/article/008/000...,,머니투데이,이병권 기자,경제,2026-02-06 11:27:37
7,ART_20260206193855,"'방산 빅4' 한화에어로-KAI, 항공엔진·무인기 등 미래사업 맞손","협력사 공급망 공유·경영진 전략위 정례화…""불필요 경쟁 탈피""\n\n\n한화에어로스...",https://n.news.naver.com/mnews/article/001/001...,,연합뉴스,홍규빈 기자,경제,2026-02-06 09:54:27
8,ART_20260206193857,"KB금융, 지난해 순이익 5조8430억…전년比 15.1%↑'역대 최대'","4분기 주당배당금 1605원 결의, 연간 배당성향 27% 기록\nKB국민은행, 작년...",https://n.news.naver.com/mnews/article/003/001...,,뉴시스,이정필 기자,경제,2026-02-05 15:56:13
9,ART_20260206193859,쿠팡만 키워주더니… 이제야 대형마트 새벽배송 ‘빗장’ 풀어주나,"당정청, 유통산업법 개정 논의\n정부와 여당이 대형마트의 ‘새벽 배송’을 금지해 온...",https://n.news.naver.com/mnews/article/023/000...,,조선일보,이미지 외 2명,경제,2026-02-06 00:51:09


In [12]:
output_filename = f"saved/Articles_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
df_articles.to_excel(output_filename, index=False, engine='openpyxl', )

In [11]:
driver.quit()